In [ ]:
# !jupyter nbconvert --to html "classifier_mel_based.ipynb"

# !jupyter nbconvert \
#      --TagRemovePreprocessor.enabled=True \
#      --TagRemovePreprocessor.remove_cell_tags="['ignore-in-py']" \
#      --to script "classifier_mel_based.ipynb" --output "./classifier_mel_based"

In [ ]:
import datetime as dt
import math
import os
from random import choice

from keras.models import load_model
import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
from sympy import *

In [ ]:
def baseline_correction(raman_spectra, lam, p, niter=10):
    i = 0
    while raman_spectra[i] == 0:
        i += 1

    # according to "Asymmetric Least Squares Smoothing" by P. Eilers and H. Boelens
    baseline_data = np.zeros(564 - i - 1)
    curr_dataset = raman_spectra[i:]
    # this is the code for the fitting procedure
    L = len(curr_dataset)
    w = np.ones(L)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    for jj in range(int(niter)):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w * curr_dataset)
        w = p * (curr_dataset > z) + (1 - p) * (curr_dataset < z)
    # end of fitting procedure

    baseline_data = z

    baseline_data = np.concatenate((np.zeros(i), baseline_data))
    return baseline_data


# kr0, kr1 = 17.354178,2.883237
ks0, ks1, ks2 = -1.627857, 1.235633, 0.0


def channel_to_energy_channel(ch, ks0, ks1, ks2):
    return ks0 + ks1 * ch + ks2 * ch ** (2)


def calibrate_spectrum_by_energy(spectrum):
    spectrum.rename(columns={0: 'channel', 1: 'counts'}, inplace=True)
    counts_orig = spectrum['counts'].values
    counts_new = np.zeros(int(channel_to_energy_channel(len(spectrum), ks0, ks1, ks2)) + 1)
    for i in range(1, len(spectrum) + 1):
        ch_ = channel_to_energy_channel(i, ks0, ks1, ks2)
        if ch_ <= int(channel_to_energy_channel(len(spectrum), ks0, ks1, ks2)) + 1 and ch_ > 20:
            K = counts_orig[i - 1]
            xv = K * (ch_ // 1 - ch_ + 1)
            counts_new[int(ch_ // 1) - 1] += xv
            counts_new[int(ch_ // 1)] += K - xv
    return counts_new


# @numba.njit
def drift_spectr(canals_orig, counts_orig, drift):
    canals_drift = np.arange(1, max(int(len(canals_orig)) + 1, int(len(canals_orig) * (1 + drift) // 1) + 2))
    counts_drift = np.zeros(max(int(len(canals_orig)), int(len(canals_orig) * (1 + drift) // 1) + 1))
    for i in range(1, len(canals_orig)):
        ch0 = canals_orig[i]
        ch_ = ch0 * (1 + drift)
        K = counts_orig[i]
        x = K * (ch_ // 1 - ch_ + 1)
        counts_drift[int(ch_ // 1) - 1] += x
        counts_drift[int(ch_ // 1)] += K - x
    if len(canals_drift) >= len(canals_orig):
        canals_drift = canals_drift[:len(canals_orig)]
        counts_drift = counts_drift[:len(canals_orig)]
    return canals_drift, np.round(counts_drift, 0)


def countNM_for_class(N, alfa, amount_in_class, min_amount_in_class):
    A = min_amount_in_class * (1 + N + alfa * N ** 2) / amount_in_class - 1
    N1 = (2 * alfa) ** (-1) * ((4 * alfa * A + 1) ** (.5) - 1)
    M1 = alfa * N1
    return int(round(N1, 0)), int(round(M1, 0))


@numba.njit
def generate_spectr(num_channels, num_quants, spectr_list):
    chnls = np.zeros(num_channels)
    for i in range(num_quants):
        target = random.random()
        array = np.asarray(spectr_list)
        idx = (np.abs(spectr_list - target)).argmin()
        num = array[idx]
        chnls[np.where(spectr_list == num)[0][-1]] += 1
    return chnls


def make_dataset1(main_dir_for_dataset):
    fldrs = [f for f in os.listdir(main_dir_for_dataset) if not f.startswith('.')]
    labels_dict = dict((k, v) for k, v in enumerate(fldrs, start=1))
    num_energies = int(channel_to_energy_channel(2048, ks0, ks1, ks2)) + 1
    data = np.empty([0, num_energies])
    labels = []
    for fldr in fldrs:
        files = [f for f in os.listdir(f'{main_dir_for_dataset}/{fldr}') if not f.startswith('.')]
        for fl in files:
            ds = pd.read_csv(f'{main_dir_for_dataset}/{fldr}/{fl}', header=None)
            ds = ds[ds[1].notna()]
            calibrated_spectrum = calibrate_spectrum_by_energy(ds)
            data = np.concatenate((data, calibrated_spectrum.reshape(1, num_energies)), axis=0)
            labels.append(list(labels_dict.keys())[list(labels_dict.values()).index(fldr)])
    datax = pd.DataFrame(data, columns=np.linspace(1, data.shape[1], data.shape[1]))
    datax['label'] = labels
    return datax, labels_dict

    # N =  количество спектров, сгенеренных из 1го с аугментацией по вертикали (симуляция измерения во времени)
    # N*alfa =  количество спектров, сгенеренных из 1го с аугментацией по горизонтали (симуляция сдвига по энергии)
    # mu - средний дрифт
    # sigma - дисперсия дрифта


def make_augmentationed_dataset_from_dataset1(N, alfa, mu, sigma, data):
    print('Begin: ', dt.datetime.now())
    M = alfa * N
    min_amount_in_class = data['label'].value_counts().min()
    canals_orig = np.array(data.columns[:-1])
    num_energies = int(channel_to_energy_channel(2048, ks0, ks1, ks2)) + 1
    dataset = np.empty([0, num_energies + 1])
    for i in range(len(data)):
        x = 1

        num_label = np.array(data.iloc[[i]])[0][-1]
        amnt_in_class = data['label'].value_counts()[num_label]
        N1, M1 = countNM_for_class(N, alfa, amnt_in_class, min_amount_in_class)
        data1 = np.zeros((1 + N1 + N1 * M1, num_energies + 1))

        data1[0] = np.array(data.iloc[[i]])[0]
        counts_orig = np.array(data.iloc[[i]])[0][:-1]
        for k in range(N1):
            rq = sum(counts_orig)
            n = int(random.random() * rq)
            dfc = np.array(counts_orig.cumsum() / sum(counts_orig))
            counts_new = generate_spectr(len(canals_orig), n, dfc)
            data1[x][:-1] = counts_new
            data1[x][-1] = num_label
            x += 1

            for j in range(M1):
                drift = np.random.normal(mu, sigma, 1)[0]
                canals_drift, counts_drift = drift_spectr(canals_orig, counts_new, drift)
                data1[x][:-1] = counts_drift
                data1[x][-1] = num_label
                x += 1

        dataset = np.concatenate((dataset, data1), axis=0)
        nuclid = labels_dict[data.loc[i]['label']]
        print(
            f'Class number {num_label} nuclide {nuclid} done!\nAdded {len(data1)} spectrums, dataset shape {dataset.shape}\n ',
            dt.datetime.now())

    return dataset


def generate_dataset_spectrum_minus_baseline(rms1, len_ds):
    ds_rms_bsln = np.zeros((len_ds, 2529))
    for e in range(len_ds):
        rms = rms1[e]
        baseline_data = baseline_correction(rms, 1000, 0.0001, niter=10)
        rms_bsln = np.array([rms[f] - baseline_data[f] if baseline_data[f] >= 0 else rms[f] for f in range(len(rms))])
        rms_bsln[rms_bsln < 0] = 0
        ds_rms_bsln[e] = rms_bsln
        if e % 1000 == 0:
            print(f'{e} from {len_ds} done! {round(e / len_ds * 100, 2)}% ', dt.datetime.now())
    return ds_rms_bsln


def mel_transform(params, E):
    s = params[0]
    a = params[4]
    m = params[5]
    b = params[3] + m * a
    c = - params[1] / m
    A = 1 / (b + a * m + 1 / m)
    B = -A * a
    C = A * c / m
    x1 = E ** .5
    return 2 * s * (B * math.log(abs(a * x1 ** 2 + b * x1 + c)) / (2 * a) + A * math.log(abs(x1 - m)) - (
            B * b - 2 * C * a) * math.atan((2 * a * x1 + b) / (4 * a * c - b * b) ** .5) / (
                            a * (4 * a * c - b * b) ** .5))


def make_mel_scale(params, kevs_to_drop):
    num_energies = int(channel_to_energy_channel(2048, ks0, ks1, ks2)) + 1
    C2 = - mel_transform(params, kevs_to_drop)
    xx = np.linspace(kevs_to_drop, num_energies, num_energies - kevs_to_drop + 1, dtype=int)
    es = np.zeros(len(xx) + kevs_to_drop)
    for E in xx:
        es[E] = mel_transform(params, E) + C2
    return es


def generate_mel_spectrum(nfilt, es, spec):
    low_mel = 0
    high_mel = es[-1]
    mel_points = np.linspace(low_mel, high_mel, nfilt + 2)
    mel_spectr = np.zeros(nfilt)
    for i in range(1, nfilt + 1):
        spectr_in_minus = np.where((es >= mel_points[i - 1]) & (es < mel_points[i]))
        spectr_in_plus = np.where((es >= mel_points[i]) & (es < mel_points[i + 1]))
        es_minus = es[spectr_in_minus]
        es_plus = es[spectr_in_plus]
        kefs_minus = np.zeros(len(spectr_in_minus[0]))
        kefs_plus = np.zeros(len(spectr_in_plus[0]))
        for g in range(len(spectr_in_minus[0])):
            kefs_minus[g] = (es_minus[g] - mel_points[i - 1]) / (mel_points[i] - mel_points[i - 1])
        for j in range(len(spectr_in_plus[0])):
            kefs_plus[j] = (mel_points[i + 1] - es_plus[j]) / (mel_points[i + 1] - mel_points[i])
        mel_spectr[i - 1] = np.dot(kefs_minus, spec[spectr_in_minus]) + np.dot(kefs_plus, spec[spectr_in_plus])
    return mel_spectr


def generate_mel_transformed_dataset(rms2):
    mel_dataset = np.zeros((len(rms2), len(mel_points) - 2))
    for i in range(len(rms2)):
        spec = rms2[i]
        mel_dataset[i] = generate_mel_spectrum(nfilt, es, spec)
        if i % 1000 == 0:
            print(f'{i} from {len(rms2)} done! {round(i / len(rms2) * 100, 2)}% ', dt.datetime.now())
    return mel_dataset


def make_mixed_spectrums_dataset(data, classes_amnt, num_sets):
    data1 = data.values
    mix_data1 = np.zeros((136 * num_sets, 2529))
    mix_labels1 = [];
    weights_comp = []
    k = 0
    for w in range(num_sets):
        for i in range(1, classes_amnt + 1):
            nomer_comp1 = choice(np.where(data1[:, -1] == i)[0])
            comp1 = data1[nomer_comp1][:-1]
            for j in range(i, classes_amnt + 1):
                nomer_comp2 = choice(np.where(data1[:, -1] == j)[0])
                comp2 = data1[nomer_comp2][:-1]
                mix_data1[k] = comp1 + comp2
                mix_labels1.append([i, j])
                weights_comp.append([sum(comp1) / (sum(comp1) + sum(comp2)), sum(comp2) / (sum(comp1) + sum(comp2))])
                k += 1
    datax1 = pd.DataFrame(mix_data1, columns=np.linspace(1, mix_data1.shape[1], mix_data1.shape[1]))
    datax1['label'] = mix_labels1
    datax1['weights'] = weights_comp
    return datax1


def make_drifted_only_dataset(datax1, Num_drifts):
    x = 0
    data1 = np.zeros((len(datax1) * (1 + Num_drifts), num_energies))
    mix_labels_list = [];
    weights_list = []
    canals_orig = np.array(datax1.columns[:-2])
    for i in range(len(datax1)):
        counts_orig = np.array(datax1.iloc[[i]])[0][:-2]
        mix_label = datax1.iloc[[i]]['label'].values[0]
        weights = datax1.iloc[[i]]['weights'].values[0]
        data1[x] = counts_orig
        mix_labels_list.append(mix_label)
        weights_list.append(weights)
        x += 1
        for j in range(Num_drifts):
            drift = np.random.normal(mu, sigma, 1)[0]
            canals_drift, counts_drift = drift_spectr(canals_orig, counts_orig, drift)
            data1[x] = counts_drift
            mix_labels_list.append(mix_label)
            weights_list.append(weights)
            x += 1
    dataset = pd.DataFrame(data1, columns=np.linspace(1, data1.shape[1], data1.shape[1]))
    dataset['label'] = mix_labels_list
    dataset['weights'] = weights_list
    return dataset


def make_mixed_spectrums_dataset_from_number_of_nuclides(nclds_in_mix, num_mixes, datav):
    mixes_vals = np.full((num_mixes, datav.shape[1] - 1), np.nan, dtype=np.float64)
    label_ind_mixes = np.full((num_mixes, classes_amnt), 0)
    for i in range(num_mixes):
        rnd_indxs = random.choice(len(datav), nclds_in_mix)
        mixes_vals[i] = np.sum(datav[rnd_indxs][:, :-1], axis=0)
        list_label_inds = datav[rnd_indxs][:, -1]
        for j in range(len(list_label_inds)):
            label_ind_mixes[i][int(list_label_inds[j] - 1)] = 1
    mixes_vals_df = pd.DataFrame(mixes_vals, columns=np.linspace(1, mixes_vals.shape[1], mixes_vals.shape[1]))
    label_ind_mixes_df = pd.DataFrame(label_ind_mixes, columns=[labels_dict[k + 1] for k in range(classes_amnt)])
    return mixes_vals_df, label_ind_mixes_df


def make_drifts_of_mixes(mixes_vals, label_ind_mixes, Num_drifts):
    x = 0
    data1 = np.zeros((len(mixes_vals) * (1 + Num_drifts), num_energies))
    labels_vektorS = np.zeros((len(mixes_vals) * (1 + Num_drifts), classes_amnt))
    canals_orig = np.linspace(1, mixes_vals.shape[1], mixes_vals.shape[1])
    for i in range(len(mixes_vals)):
        counts_orig = mixes_vals[i]
        labels_vektorS[x] = label_ind_mixes[i]
        data1[x] = counts_orig
        x += 1
        for j in range(Num_drifts):
            drift = np.random.normal(mu, sigma, 1)[0]
            canals_drift, counts_drift = drift_spectr(canals_orig, counts_orig, drift)
            data1[x] = counts_drift
            labels_vektorS[x] = label_ind_mixes[i]
            x += 1
    data1_df = pd.DataFrame(data1, columns=np.linspace(1, mixes_vals.shape[1], mixes_vals.shape[1]))
    labels_vektorS_df = pd.DataFrame(labels_vektorS, columns=[labels_dict[k + 1] for k in range(classes_amnt)])
    df = pd.concat([data1_df, labels_vektorS_df], axis=1)
    return df


def generate_random_weights(n):
    if n == 1:
        return [1]
    else:
        zzz = np.sort(np.random.random(n - 1))
        if n == 2:
            return [zzz[0], 1 - zzz[0]]
        else:
            zzzr = [zzz[0]]
            for i in range(len(zzz) - 1):
                zzzr.append(zzz[i + 1] - zzz[i])
            zzzr.append(1 - zzz[i + 1])
            return zzzr


def gen_smix(num_nclds_in_mix):
    randm_ncld_clss_lst = np.random.choice(ncldsss_lst, num_nclds_in_mix, replace=False)
    wt = generate_random_weights(num_nclds_in_mix)
    smix0 = np.zeros(num_energies)
    scnts0 = np.zeros(num_nclds_in_mix)
    for i in range(num_nclds_in_mix):
        smix = data_mono[data_mono['nuclide'] == randm_ncld_clss_lst[i]].values
        smix = smix[int(np.random.choice(np.linspace(0, len(smix) - 1, len(smix))))][:-1]
        smix0 = smix0 + smix * wt[i]
        scnts0[i] = sum(smix * wt[i])
    wtF = scnts0 / sum(scnts0)
    return randm_ncld_clss_lst, wtF, smix0


## Чтение сохраненных моделей и их анализ.


In [ ]:
%%time
nfilt = int(141*4)
q1,q2,q3,q4=-4.972648,3.076921,-0.050956,0.000696
s = 10
kevs_to_drop = 20
x = symbols('x')
m = solve(q1 + q2*x + q3*x**(2) + q4*x**(3), x)[0]
params = np.array([s,q1,q2,q3,q4,m])
es = make_mel_scale(params,kevs_to_drop)
low_mel = 0
high_mel = es[-1]
mel_points = np.linspace(low_mel, high_mel, nfilt + 2)
fig,ax = plt.subplots(figsize=(30,10))
ax.plot(np.linspace(0,2529,2530),es)
ax.grid(True)

In [ ]:
%%time
main_dir_for_dataset = './GammaSpectraGeant4/'

N,alfa,mu,sigma = 5, 1, 0, 0.01
Num_drifts = 4
classes_amnt = 16
num_sets = 33
num_energies = int(channel_to_energy_channel(2048,ks0, ks1, ks2)) + 1
data ,labels_dict= make_dataset1(main_dir_for_dataset)

In [ ]:
data_mono = data.copy()
data_mono['nuclide'] = [labels_dict[k] for k in data_mono['label'].values]
data_mono.drop(columns='label',inplace=True)
data_mono

In [ ]:
ncldsss_lst = np.unique(data_mono['nuclide'])
ncldsss_lst

In [ ]:
saved_model_Am_241 = load_model(f'./classification_models_mononuclides/best_model_Am-241.h5')
saved_model_Ce_141 = load_model(f'./classification_models_mononuclides/best_model_Ce-141.h5')
saved_model_Ce_144 = load_model(f'./classification_models_mononuclides/best_model_Ce-144.h5')
saved_model_Co_58 = load_model(f'./classification_models_mononuclides/best_model_Co-58.h5')
saved_model_Co_60 = load_model(f'./classification_models_mononuclides/best_model_Co-60.h5')
saved_model_Cr_51 = load_model(f'./classification_models_mononuclides/best_model_Cr-51.h5')
saved_model_Cs_134 = load_model(f'./classification_models_mononuclides/best_model_Cs-134.h5')
saved_model_Cs_137 = load_model(f'./classification_models_mononuclides/best_model_Cs-137.h5')
saved_model_Eu_152 = load_model(f'./classification_models_mononuclides/best_model_Eu-152.h5')
saved_model_Fe_59 = load_model(f'./classification_models_mononuclides/best_model_Fe-59.h5')
saved_model_I_131 = load_model(f'./classification_models_mononuclides/best_model_I-131.h5')
saved_model_Mn_54 = load_model(f'./classification_models_mononuclides/best_model_Mn-54.h5')
saved_model_Ru_103 = load_model(f'./classification_models_mononuclides/best_model_Ru-103.h5')
saved_model_Ru_106 = load_model(f'./classification_models_mononuclides/best_model_Ru-106.h5')
saved_model_Zn_65 = load_model(f'./classification_models_mononuclides/best_model_Zn-65.h5')
saved_model_Zr_95 = load_model(f'./classification_models_mononuclides/best_model_Zr-95.h5')

# saved_model_Am_241 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Am-241.h5')
# saved_model_Ce_141 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Ce-141.h5')
# saved_model_Ce_144 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Ce-144.h5')
# saved_model_Co_58 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Co-58.h5')
# saved_model_Co_60 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Co-60.h5')
# saved_model_Cr_51 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Cr-51.h5')
# saved_model_Cs_134 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Cs-134.h5')
# saved_model_Cs_137 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Cs-137.h5')
# saved_model_Eu_152 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Eu-152.h5')
# saved_model_Fe_59 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Fe-59.h5')
# saved_model_I_131 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_I-131.h5')
# saved_model_Mn_54 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Mn-54.h5')
# saved_model_Ru_103 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Ru-103.h5')
# saved_model_Ru_106 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Ru-106.h5')
# saved_model_Zn_65 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Zn-65.h5')
# saved_model_Zr_95 = load_model(f'./classification_models_mononuclides/best_model_melBs_3mix_Zr-95.h5')

In [ ]:
ncldsss_lst = np.unique(data_mono['nuclide'])
comps,wesa,smix = gen_smix(2)

weights_list = np.in1d(ncldsss_lst,comps).astype(float)
w_true_list = np.zeros(len(ncldsss_lst))
w_true_list[np.where(weights_list == 1)] = wesa

drift = np.random.normal(mu, sigma, 1)[0]
canals_orig = np.linspace(1,smix.shape[0],smix.shape[0])

smes = drift_spectr(canals_orig, smix,drift)[1]
#smes_log = np.log10(smes+1)/sum(np.log10(smes+1))
mel_mix = generate_mel_spectrum(nfilt,es,smes)
rmspec = np.log10(mel_mix + 1)
# bsln = baseline_correction(rmspec,10,0.0001,niter=10)
# bsln = np.where(bsln<0,0,bsln)
# res = np.where((rmspec - bsln) < 0, 0, (rmspec - bsln))

rms_bsln_o = rmspec/sum(rmspec)

preds_list = np.zeros(len(ncldsss_lst))
# preds_list[10] = saved_model_I_131.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[6] = saved_model_Cs_134.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[12] = saved_model_Ru_103.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[15] = saved_model_Zr_95.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[5] = saved_model_Cr_51.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
preds_list[0] = saved_model_Am_241.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
preds_list[4] = saved_model_Ru_106.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[1] = saved_model_Ce_141.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[2] = saved_model_Ce_144.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
preds_list[3] = saved_model_Eu_152.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
preds_list[2] = saved_model_Cs_137.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
preds_list[1] = saved_model_Co_60.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[11] = saved_model_Mn_54.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[3] = saved_model_Co_58.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[9] = saved_model_Fe_59.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]
# preds_list[14] = saved_model_Zn_65.predict(rms_bsln_o.reshape((1,564)),batch_size = 1,verbose=0)[0][0]

verificat_df = pd.DataFrame({'Nuclide':ncldsss_lst,'weight':weights_list,'true_weight':w_true_list,'predictions':preds_list}).T
display(verificat_df)